<a href="https://colab.research.google.com/github/JorgeGalarza777/SIM_YAC_GALARZA_UPSE2023/blob/main/METODOEXPLICITOEIMPLICITO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MÉTODOS EXPLÍCITOS E IMPLÍCITOS DE LA ECUACIÓN DE LA DIFUSIVIDAD**

In [ ]:
import numpy as np
import math

In [ ]:
L=10000
dx=2500
Tiempo=100000
DT=1
K=50 #permeabilidad MD
por=0.2 #porosidad
vis=1 #centipoise
Beta=1
CompTotal=1**-6 #ps1-1
A=200000 #ft^2


In [ ]:
Alpha=K/(vis*por*CompTotal)
Alpha
Eta=(Alpha*DT)/(dx**2)
Eta

4e-05

In [ ]:
#Esquema Explicito
def Presdt(P0,P1,P2,Eta):
  Pdt=P1+Eta*(P0-2*P1+P2)
  return Pdt

Pexplicita=np.zeros((Tiempo//DT,L//dx))

In [ ]:
#Condiciones Iniciales
Pyac=1000 #psi
Piz=2000 #psi
Pder=0 #psi
Pin=np.ones(L//dx)
Pres=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz
Pin[len(Pin)-1]=Pder

Pin

array([2000., 1000., 1000.,    0.])

In [ ]:
t=DT
Pexplicita[0]=Pin
Pexplicita.shape
while t<Tiempo:
  for i in range(0,len(Pin)-1):
    if i==0:
      Pres[i]=Pin[0]
    elif i==len(Pin)-1:
      Pres[i]=Pin[len(Pin)-1]
    else:
      Pres[i]=Presdt(Pin[i-1],Pin[i],Pin[i+1],Eta)
  Pexplicita[t//DT]=Pres
  t=t+DT
  Pin=Pres

In [ ]:
Pexplicita

array([[2000.        , 1000.        , 1000.        ,    0.        ],
       [2000.        , 1000.04      ,  999.96      ,    0.        ],
       [2000.        , 1000.0799952 ,  999.9200064 ,    0.        ],
       ...,
       [2000.        , 1333.33146859,  666.66889757,    0.        ],
       [2000.        , 1333.33146883,  666.66889732,    0.        ],
       [2000.        , 1333.33146907,  666.66889706,    0.        ]])

In [56]:
#Método Implicito
Pexplicita=np.zeros((Tiempo//DT,L//dx))
Pimplicita=np.zeros((Tiempo//DT,L//dx))

Pin=np.ones(L//dx)
Pres=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz
Pin[len(Pin)-1]=Pder
N=L//dx
Matriz_resolucion=np.zeros([N-2,N-2])

In [57]:
for i in range(N-2): #se pone -2 porque es matriz de resolución
  Matriz_resolucion[i,i]=1+2*Eta
  if i>0:
    Matriz_resolucion[i,i-1]=-Eta
  if i<(N-3):
    Matriz_resolucion[i,i+1]=-Eta

In [58]:
Pcal=np.zeros([N-2])
Pcal=np.linalg.solve(Matriz_resolucion,Pin[1:-1])

In [59]:
Pcal

array([999.9600016, 999.9600016])

In [60]:
Pres=np.concatenate(([Piz],Pcal,[Pder]))
Pres

array([2000.       ,  999.9600016,  999.9600016,    0.       ])

In [61]:
Pimplicita[0]=Pin
while t<Tiempo:
  Pcal=np.linalg.solve(Matriz_resolucion,Pin[1:-1])
  Pres=np.concatenate(([Piz],Pcal,[Pder]))
  Pimplicita[t//DT]=Pres
  Pin=Pres
  t=t+DT

In [62]:
Pimplicita

array([[2000., 1000., 1000.,    0.],
       [   0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.]])